****************************************************************************
* @file: DNN.py
* @project: ROBOFEI-HT - FEI 😛
* @author: Vinicius Nicassio Ferreira
* @version: V0.0.1
* @created: 23/10/2017
* @e-mail: vinicius.nicassio@gmail.com
* @brief: Class DNN
****************************************************************************

In [1]:
#ini-iPython

# Executando no diretório principal

In [2]:
import os
os.chdir('../../') #Executando no diretório principal
os.system('cd Workbench/ && ./zerar.sh && cd ..')
os.system('cd Workbench/iPython/ && ./convertendo.sh && cd ../../')
import sys
sys.path.append('./include')
sys.path.append('./src')
sys.path.append('./Workbench/iPython')
#end-iPython

---- Imports ----

In [3]:
# The standard libraries used in the vision system.
import tarfile # Used for manipulating tar files.
import pandas as pd # Utilizado para manipular dados e tabelas
import os # Biblioteca utilizada para interação com o sistema
import shutil # Biblioteca utilizada para apagar diretorios com arquivos
import time # Libraries used for time management.

In [4]:
# The standard libraries used in the visual memory system.
import cv2 # OpenCV library used for image processing.
import numpy as np # Library used for operations with matrix and array.

# Libraries to supporting the network execution using Tensorflow.
#ini-iPython - Utilizado para o iPython reconhecer as classes
sys.path.append('/home/vinicius/models/research')
sys.path.append('/home/vinicius/models/research/object_detection')
#end-iPython
import tensorflow as tf
from utils import label_map_util
from utils import visualization_utils as vis_util

In [5]:
# Used class developed by RoboFEI-HT.
#ini-iPython - Usando Classe BasicProcesses como instanciação
os.system("sed -i '/@abstractmethod/d' ./Workbench/iPython/BasicProcesses.py")
os.system("sed -i '/__metaclass__/d' ./Workbench/iPython/BasicProcesses.py")
#end-iPython
from BasicProcesses import * # Standard and abstract class.

# Class DNN
Class that implements object detection using a deep neural network (DNN).
doxygen-iPython

In [6]:
#des-iPython class DNN(BasicProcesses):
#ini-tab -> Iniciando código classe

---- Variables ----

In [7]:
#ini-iPython

# ------- Lista de funções -------

#self-iPython __unzipNetwork
#self-iPython __instantiatingDNNVariables
#self-iPython __detectDNN
#self-iPython detect
#self-iPython finalize
#self-iPython __trackbarThresholdMin
#self-iPython __trackbarThresholdMax

class argumentos:
    camera = False
    dnn = True
    train = False

a = argumentos()
#end-iPython - Fim listas

# EXTRACTION_DIRECTORY
Diretorio aonde será extraida a rede.
doxygen-iPython

In [8]:
__EXTRACTION_DIRECTORY = './Data/Rede' #self-iPython __EXTRACTION_DIRECTORY

# PATH_TO_CKPT
Path to frozen detection graph. This is the actual model that is used for the object detection.
doxygen-iPython

In [9]:
__PATH_TO_CKPT = 'frozen_inference_graph.pb' #self-iPython __PATH_TO_CKPT

# PATH_TO_LABELS
List of the strings that is used to add correct label for each box.
doxygen-iPython

In [10]:
__PATH_TO_LABELS = 'object-detection.pbtxt' #self-iPython __PATH_TO_LABELS

# DIRECTORY_TRAINING_IMAGES
Directory where the training imagens will be saved.
doxygen-iPython

In [11]:
__DIRECTORY_TRAINING_IMAGES = './Train' #self-iPython __DIRECTORY_TRAINING_IMAGES

# numclasses
The Number of classes that are detected by the network.
doxygen-iPython

In [12]:
__numclasses = None #self-iPython __numclasses

# detection_graph
Variavel de configuração do TensorFlow.
doxygen-iPython

In [13]:
__detection_graph = None #self-iPython __detection_graph

# sess
Variavel de execução do TensorFlow.
doxygen-iPython

In [14]:
__sess = None #self-iPython __sess

# __label
Dicionario com as strings das classes que seram detectadas pela rede.
doxygen-iPython

In [15]:
__label = None #self-iPython __label

# __parameters
Parametros de configuração para a execução da DNN.
doxygen-iPython

In [16]:
__parameters = None #self-iPython __parameters

Variaveis de apoio do TensorFlow

In [17]:
__imagetensor = __detectionboxes = __detectionscores = __detectionclasses = __numdetections = __category_index = None
#ini-iPython
#self-iPython __imagetensor
#self-iPython __detectionboxes
#self-iPython __detectionscores
#self-iPython __detectionclasses
#self-iPython __numdetections
#self-iPython __category_index
#end-iPython

# __unzipNetwork
.
doxygen-iPython

In [18]:
#eself-iPython __unzipNetwork
#edes-iPython def __unzipNetwork(self, name):
def __unzipNetwork(name):
    tar = tarfile.open('./Data/'+name+'.tar.gz')
    tar.extractall(path=__EXTRACTION_DIRECTORY)
    tar.close() #self-iPython __unzipNetwork

In [19]:
#ini-iPython

## Teste __unzipNetwork

In [20]:
__unzipNetwork('rede')
#end-iPython

# instantiatingDNNVariables
Used to create/instantiate the variables that will be used by DNN in detecting objects.
doxygen-iPython

In [21]:
#eself-iPython __instantiatingDNNVariables
#edes-iPython def __instantiatingDNNVariables(self):
def __instantiatingDNNVariables( ):
    global __numclasses, __detection_graph, __sess, __label #debug-iPython
    global __imagetensor, __detectionboxes, __detectionscores, __detectionclasses, __numdetections, __category_index #debug-iPython
    # Reading network file.
    __detection_graph = tf.Graph()
    with __detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(__EXTRACTION_DIRECTORY+ '/' +__PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    label_map = label_map_util.load_labelmap(__EXTRACTION_DIRECTORY+ '/' +__PATH_TO_LABELS)
    
    filelabel = str(label_map).replace('\n', '').replace(' ', '').replace('{', '').replace('}', '').replace('\"', '\'')
    __label = {}
    print 'Todas classes:', filelabel.split('item')[1:] #debug-iPython
    print '\n' #debug-iPython
    for text in filelabel.split('item')[1:]:
        print 'text:', text #debug-iPython
        key = float(text.split(':')[-1])
        print 'key:', key #debug-iPython
        print 'name:', text.split('\'')[1] #debug-iPython
        name = text.split('\'')[1]
        __label[key] = name
        print '\n' #debug-iPython
    print __label #debug-iPython
    
    if self._args.dnn == True:
        __numclasses = str(label_map).count('id')
        categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=__numclasses, use_display_name=True)
        __category_index = label_map_util.create_category_index(categories)

    # Creating a section to run the detection.
    with __detection_graph.as_default():
        __sess = tf.Session(
            graph=__detection_graph,
            config=tf.ConfigProto(
                intra_op_parallelism_threads=1,
                inter_op_parallelism_threads=1
            )
        )
        
        __imagetensor = __detection_graph.get_tensor_by_name('image_tensor:0')
        __detectionboxes = __detection_graph.get_tensor_by_name('detection_boxes:0')
        __detectionscores = __detection_graph.get_tensor_by_name('detection_scores:0')
        __detectionclasses = __detection_graph.get_tensor_by_name('detection_classes:0')
        __numdetections = __detection_graph.get_tensor_by_name('num_detections:0') #self-iPython __instantiatingDNNVariables

In [22]:
#ini-iPython
__label

## Teste __instantiatingDNNVariables

In [23]:
self = BasicProcesses(a, 'DNN', 'Parameters')
__instantiatingDNNVariables()
__numclasses, __detection_graph, __sess

DNN - PARAMETERS section not found!
Todas classes: ["name:'ball'id:1", "name:'robot'id:2"]


text: name:'ball'id:1
key: 1.0
name: ball


text: name:'robot'id:2
key: 2.0
name: robot


{1.0: 'ball', 2.0: 'robot'}


(2,
 <tensorflow.python.client.session.Session at 0x7ff2d32b5290>)

In [24]:
__numclasses = __detection_graph = __sess = self = None
#end-iPython

# trackbarThresholdMin
.
doxygen-iPython

In [25]:
#eself-iPython __trackbarThresholdMin
#edes-iPython def __trackbarThresholdMin(self, value):
def __trackbarThresholdMin(value):
    __parameters['threshold_to_train_min'] = min(value/100.0, __parameters['threshold_to_train_max']-0.01)
    cv2.setTrackbarPos('threshold_min', 'DNN - Parameters', int(__parameters['threshold_to_train_min']*100)) #self-iPython __trackbarThresholdMin

# trackbarThresholdMax
.
doxygen-iPython

In [26]:
#eself-iPython __trackbarThresholdMax
#edes-iPython def __trackbarThresholdMax(self, value):
def __trackbarThresholdMax(value):
    __parameters['threshold_to_train_max'] = max(value/100.0, __parameters['threshold_to_train_min']+0.01)
    cv2.setTrackbarPos('threshold_max', 'DNN - Parameters', int(__parameters['threshold_to_train_max']*100)) #self-iPython __trackbarThresholdMax

# Constructor Class
doxygen-iPython

In [27]:
#des-iPython def __init__(self, a):
#ini-tab -> Criando construtor
#edes-iPython super(DNN, self).__init__(a, 'DNN', 'Parameters')
self = BasicProcesses(a, 'DNN', 'Parameters')

# Creating and updating default parameter values.
__parameters = {
    'network_name': 'rede',
    'threshold_to_train_min': 0.2,
    'threshold_to_train_max': 0.5,
}
__parameters = self._conf.readVariables(__parameters)

# Creating neural network.
__unzipNetwork(__parameters['network_name'])
__instantiatingDNNVariables()

if self._args.dnn == True:
    cv2.namedWindow('DNN - Parameters')
    cv2.createTrackbar(
        'threshold_min',
        'DNN - Parameters',
        int(__parameters['threshold_to_train_min']*100),
        100,
        __trackbarThresholdMin
    )
    cv2.createTrackbar(
        'threshold_max',
        'DNN - Parameters',
        int(__parameters['threshold_to_train_max']*100),
        100,
        __trackbarThresholdMax
    )

if self._args.train == True:
    try:
        os.makedirs(__DIRECTORY_TRAINING_IMAGES+'/imagens to classify')
    except OSError:
        pass
    try:
        os.makedirs(__DIRECTORY_TRAINING_IMAGES+'/annotations DNN')
    except OSError:
        pass

DNN - PARAMETERS section not found!
Todas classes: ["name:'ball'id:1", "name:'robot'id:2"]


text: name:'ball'id:1
key: 1.0
name: ball


text: name:'robot'id:2
key: 2.0
name: robot


{1.0: 'ball', 2.0: 'robot'}


In [28]:
#end-tab -> Finalizando construtor

In [29]:
#ini-iPython

## Teste construtor

In [30]:
__numclasses, __detection_graph, __sess
#end-iPython

(2,
 <tensorflow.python.client.session.Session at 0x7ff318a33050>)

# detectDNN
.
doxygen-iPython

In [31]:
#eself-iPython __detectDNN
#edes-iPython def __detectDNN(self, img):
def __detectDNN(img):
    global __sess #debug-iPython
    # Actual detection.
    image_np_expanded = np.expand_dims(img, axis=0)
    (boxes, scores, classes, num) = __sess.run(
        [__detectionboxes, __detectionscores, __detectionclasses, __numdetections],
        feed_dict={__imagetensor: image_np_expanded}
    )
    
    if self._args.dnn == True:
        # Visualization of the results of a detection.
        img = cv2.resize(
            img,
            None,
            fx=380.0/img.shape[0],
            fy=380.0/img.shape[0]
        )
        vis_util.visualize_boxes_and_labels_on_image_array(
            img,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            __category_index,
            use_normalized_coordinates=True,
            line_thickness=2,
            min_score_thresh=__parameters['threshold_to_train_min'],
        )
    
    # Creating DataFrame detection.
    classes = classes.tolist()
    for i in xrange(len(classes[0])):
        classes[0][i] = __label[classes[0][i]]
    
    df = pd.DataFrame()
    df['classes'] = classes[0]
    df['scores'] = scores[0]
    df['boxes'] = boxes[0].tolist()
    
    return df, img #self-iPython __detectDNN

In [32]:
#ini-iPython

## Teste detectDNN

In [33]:
img = cv2.imread('/home/vinicius/objectDetect_clear/Train/imagenstreino/frames0_040.jpg')
df, img = __detectDNN(img)
cv2.imshow('Frame 2', img)
cv2.waitKey(0)
cv2.destroyWindow('Frame 2')
df
#end-iPython

,classes,scores,boxes
0,robot,0.983122,"[0.304917991161, 0.896200180054, 0.56376618146..."
1,robot,0.948715,"[0.21794205904, 0.517218530178, 0.711023211479..."
2,ball,0.870266,"[0.449643224478, 0.364045917988, 0.54780566692..."
3,robot,0.006161,"[0.100663363934, 0.983142375946, 0.35967051982..."
4,ball,0.005959,"[0.300438791513, 0.892324090004, 0.56336617469..."
5,robot,0.003821,"[0.63169491291, 0.72311937809, 0.999579191208,..."
6,robot,0.002172,"[0.449643224478, 0.364045917988, 0.54780566692..."
7,robot,0.002050,"[0.217791080475, 0.00317733734846, 0.924235701..."
8,robot,0.001631,"[0.238770037889, 0.0111953765154, 0.9402588605..."
9,robot,0.001551,"[0.0119702517986, 0.00471384823322, 0.79037737..."


# detect
.
doxygen-iPython

In [34]:
#eself-iPython detect
#edes-iPython def detect(self, observation):
def detect(observation):
    if self._args.train == True:
        image = observation['frame'].copy()
    
    objects, observation['frame'] = __detectDNN(observation['frame'])
    
    if self._args.dnn == True:
        cv2.imshow('DNN - Parameters', observation['frame'])
        if cv2.waitKey(1) == ord('q'):
            raise VisionException(5, 'DNN')
            
    if self._args.train == True:
        cv2.imwrite(__DIRECTORY_TRAINING_IMAGES+'/imagens to classify/'+'Robot-'+ time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(observation['time']))) + str(round(observation['time'] - int(observation['time']), 2))[1:] +'.jpg', image)

        if len(objects.scores) != 0:        
            text = '<annotation>\n\t<folder>imagens to classify</folder>\n\t<filename>Robot-'+ time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(observation['time']))) + str(round(observation['time'] - int(observation['time']), 2))[1:] +'.jpg</filename>\n\t<path>'+ os.getcwd() +'/'+ __DIRECTORY_TRAINING_IMAGES.split('/')[-1] +'/imagens to check/Robot-'+ time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(observation['time']))) + str(round(observation['time'] - int(observation['time']), 2))[1:] +'.jpg</path>\n\t<source>\n\t\t<database>Unknown</database>\n\t</source>\n\t<size>\n\t\t<width>'+ str(image.shape[1]) +'</width>\n\t\t<height>'+ str(image.shape[0]) +'</height>\n\t\t<depth>'+ str(image.shape[2]) +'</depth>\n\t</size>\n\t<segmented>0</segmented>'

            for classes, __, box in objects[(objects.scores > __parameters['threshold_to_train_min'])].values:
                text += '\n\t<object>\n\t\t<name>'+ classes+ '</name>\n\t\t<pose>Unspecified</pose>\n\t\t<truncated>0</truncated>\n\t\t<difficult>0</difficult>\n\t\t<bndbox>\n\t\t\t<xmin>'+ str(int(box[1]*image.shape[1])) +'</xmin>\n\t\t\t<ymin>'+ str(int(box[0]*image.shape[0])) +'</ymin>\n\t\t\t<xmax>'+ str(int(box[3]*image.shape[1])) +'</xmax>\n\t\t\t<ymax>'+ str(int(box[2]*image.shape[0])) +'</ymax>\n\t\t</bndbox>\n\t</object>'

            text += '\n</annotation>'

            f = open(__DIRECTORY_TRAINING_IMAGES + '/annotations DNN/' + 'Robot-'+ time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(observation['time']))) + str(round(observation['time'] - int(observation['time']), 2))[1:] +'.xml', 'w')
            f.write(text)
            f.close()
    
    return objects[objects.scores > __parameters['threshold_to_train_max']] #self-iPython detect

In [35]:
#ini-iPython

## Teste detect

In [36]:
observation = {}
while True:
    try:
        observation['time'] = time.time()
        observation['frame'] = cv2.imread('/home/vinicius/objectDetect_clear/Train/imagenstreino/frames0_040.jpg')
#         observation['frame'][:,:,:] = 0
        objects = detect(observation)
    except VisionException as e:
        break
#end-iPython

Solicitação encerramento de processo pela thread 


# finalize
.
doxygen-iPython

In [37]:
#eself-iPython finalize
#edes-iPython def finalize(self):
def finalize( ):
    # Deleting extracted network
    try:
        shutil.rmtree('./Data/Rede')
    except OSError:
        pass
    
    if self._args.dnn == True:
        cv2.destroyAllWindows()
    
    # Saving parameter values
    self._end() #self-iPython finalize

In [38]:
#ini-iPython

## Teste finalize

In [39]:
finalize()
#end-iPython

In [40]:
#end-tab -> Finalizando classe

In [41]:
#ini-iPython

# nova_variavel
.
doxygen-iPython

In [42]:
nova_variavel = None #self-iPython nova_variavel

# nova_funcao
.
doxygen-iPython

In [43]:
# eself-iPython nova_funcao
# edes-iPython def nova_funcao(self, variaveis_funcao):
def nova_funcao(variaveis_funcao):
    pass # self-iPython nova_funcao

In [44]:
#ini-iPython

## Teste nova_funcao

In [45]:

#end-iPython

In [46]:
#end-iPython